In [ ]:
# Cell 1: Clone repo and install dependencies
import os

REPO_URL = "https://github.com/elonmj/Code-traffic-flow.git"
REPO_DIR = "/content/Code-traffic-flow"

if os.path.exists(REPO_DIR):
    print(f"Repository already exists at {REPO_DIR}")
    %cd {REPO_DIR}
    !git pull
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

!pip install stable-baselines3 gymnasium numba --quiet
print(f"✅ Setup complete | Working dir: {os.getcwd()}")

Cloning into '/content/Code-traffic-flow'...
remote: Enumerating objects: 9009, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 9009 (delta 123), reused 169 (delta 64), pack-reused 8772 (from 2)
Receiving objects: 100% (9009/9009), 317.35 MiB | 25.25 MiB/s, done.
Resolving deltas: 100% (4619/4619), done.
/content/Code-traffic-flow
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 7.2 MB/s eta 0:00:00
✅ Setup complete | Working dir: /content/Code-traffic-flow


In [ ]:
# Cell 2: Imports & Setup
import sys
sys.path.insert(0, REPO_DIR)

import numpy as np
import torch
import time
import os
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.callbacks import BaseCallback

from arz_model.config import create_victoria_island_config
from Code_RL.src.env.traffic_signal_env_direct_v3 import TrafficSignalEnvDirectV3

# Mount Google Drive for persistence
try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = "/content/drive/MyDrive/thesis_runs_stage3_ppo"
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"✅ Google Drive mounted. Saving results to: {SAVE_DIR}")
except:
    SAVE_DIR = "/content"
    print(f"⚠️ Google Drive not available. Saving to: {SAVE_DIR}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PyTorch: 2.9.0+cu126
CUDA: True
GPU: Tesla T4


In [4]:
# Cell 3: Configuration
# FIX: Reduce phase change penalty to encourage exploration
DEFAULT_DENSITY = 120.0  # Congested scenario
INFLOW_DENSITY = 180.0

# IMPORTANT: Set kappa=0.0 to remove phase change penalty initially
# This allows the agent to explore switching without penalty
REWARD_WEIGHTS = {'alpha': 5.0, 'kappa': 0.0, 'mu': 0.1}  # kappa=0.0 removes switch penalty

def create_env(quiet=True):
    """Create traffic environment"""
    config = create_victoria_island_config(
        t_final=450.0, output_dt=15.0, cells_per_100m=4,
        default_density=DEFAULT_DENSITY, inflow_density=INFLOW_DENSITY, use_cache=False
    )
    config.rl_metadata = {'observation_segment_ids': [s.id for s in config.segments], 'decision_interval': 15.0}
    
    class SimpleConfig:
        def __init__(self, c):
            self.arz_simulation_config = c
            self.rl_env_params = {'dt_decision': 15.0, 'observation_segment_ids': None, 'reward_weights': REWARD_WEIGHTS}
    
    return TrafficSignalEnvDirectV3(
        simulation_config=SimpleConfig(config).arz_simulation_config,
        decision_interval=15.0, reward_weights=REWARD_WEIGHTS, quiet=quiet
    )

print(f"✅ Environment ready | kappa={REWARD_WEIGHTS['kappa']} (no switch penalty)")
print(f"   Congestion: alpha={REWARD_WEIGHTS['alpha']}, Throughput: mu={REWARD_WEIGHTS['mu']}")

✅ Environment ready | kappa=0.0 (no switch penalty)
   Congestion: alpha=5.0, Throughput: mu=0.1


In [5]:
# Cell 4: Evaluate Baselines (Random, FT-30s, FT-60s, FT-90s)
env = create_env()
baseline_results = {}

def eval_fixed_time(env, interval, n_ep=3):
    rewards = []
    for _ in range(n_ep):
        obs, _ = env.reset()
        done, ep_r, t = False, 0.0, 0.0
        while not done:
            t += env.decision_interval
            action = 1 if t >= interval else 0
            if action == 1: t = 0.0
            obs, r, done, _, _ = env.step(action)
            ep_r += r
        rewards.append(ep_r)
    return {'mean_reward': np.mean(rewards), 'std_reward': np.std(rewards)}

# Random
print("🎲 Random...")
rnd = [sum([env.step(env.action_space.sample())[1] for _ in range(30)]) for _ in [env.reset() for _ in range(3)]]
baseline_results['Random'] = {'mean_reward': np.mean(rnd), 'std_reward': np.std(rnd)}

# Fixed-time baselines
for name, interval in [('FT-30s', 30), ('FT-60s', 60), ('FT-90s', 90)]:
    print(f"⏱️ {name}...")
    baseline_results[name] = eval_fixed_time(env, interval)

print("\n📊 BASELINES:")
for n, d in sorted(baseline_results.items(), key=lambda x: x[1]['mean_reward'], reverse=True):
    print(f"  {n:10s}: {d['mean_reward']:>8.1f} ± {d['std_reward']:.1f}")


🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   🚦 Detected 8 signalized nodes from OSM data

   🔧 Generating segment configurations...
   ✅ Created 70 segment configurations

   🔧 Generating node configurations...
   ✅ Created 60 node configurations

   ⚙️  Setting up time and physics parameters...

   🔨 Assembling complete network configuration...

✅ CONFIGURATION GENERATION COMPLETE
   Total Segments: 70
   Total Nodes: 60
   Entry Points: 4
   Exit Points: 4
   Junctions: 15
   Signalized Nodes: 8
   Simulation Duration: 450.0s (7.5 min)
   Grid Resolution: 4 cells/100m

[NETWORK BUILD] Creating 70 segments and 60 nodes...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


   [10/70] segments created
   [20/70] segments created
   [30/70] segments created
   [40/70] segments created
   [50/70] segments created
   [60/70] segments created
   [70/70] segments created
[NETWORK BUILD] ✅ Network construction complete!
   Total segments: 70
   Total nodes: 60
Finalizing network structure and validating topology...
✅ Network topology is valid.


...

  - Preparing GPU topology for network coupling...
    - GPU topology prepared and transferred.
🎲 Random...


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/

⏱️ FT-30s...


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid siz

⏱️ FT-60s...
⏱️ FT-90s...
⏱️ FT-90s...

📊 BASELINES:
  FT-60s    :    799.7 ± 0.0
  FT-90s    :    799.6 ± 0.0
  Random    :    794.3 ± 36.0
  FT-30s    :    792.1 ± 10.6

📊 BASELINES:
  FT-60s    :    799.7 ± 0.0
  FT-90s    :    799.6 ± 0.0
  Random    :    794.3 ± 36.0
  FT-30s    :    792.1 ± 10.6


### 🧠 Stage 3 - PPO Training (Alternative to DQN)

This notebook uses **PPO (Proximal Policy Optimization)** instead of DQN.

**Key differences:**
- **PPO**: On-policy, policy gradient method. Better at exploration via entropy regularization.
- **DQN**: Off-policy, Q-learning method. Uses replay buffer for sample efficiency.

**Hypothesis**: PPO may converge faster or find better policies due to its inherent exploration mechanism (entropy bonus).

In [ ]:
# Cell 5: Progressive Training Callback
class ProgressCallback(BaseCallback):
    def __init__(self, eval_env, ref_reward, target_pct=10.0, eval_freq=1000, n_eval=3):
        super().__init__()
        self.eval_env, self.ref = eval_env, ref_reward
        self.target = ref_reward * (1 + target_pct/100)
        self.eval_freq, self.n_eval = eval_freq, n_eval
        self.history, self.best, self.reached = [], -np.inf, False
        
    def _on_step(self):
        if self.n_calls % self.eval_freq == 0:
            rewards = []
            all_actions = []  # Track actions across episodes
            for _ in range(self.n_eval):
                obs, done, r = self.eval_env.reset()[0], False, 0
                ep_actions = []
                while not done:
                    a, _ = self.model.predict(obs, deterministic=True)
                    ep_actions.append(int(a))
                    obs, rew, done, _, _ = self.eval_env.step(a)
                    r += rew
                rewards.append(r)
                all_actions.append(ep_actions)
            
            mean_r = np.mean(rewards)
            imp = ((mean_r - self.ref) / abs(self.ref)) * 100
            self.history.append({'step': self.num_timesteps, 'reward': mean_r, 'improvement': imp})
            if mean_r > self.best: self.best = mean_r
            
            # Analyze actions
            avg_actions = all_actions[0]  # Use first episode as representative
            n_switches = sum(avg_actions)
            n_steps = len(avg_actions)
            switch_rate = n_switches / n_steps * 100
            
            status = "🎯" if imp >= 10 else ""
            print(f"  [{self.num_timesteps:>6}] R={mean_r:>7.1f} | vs FT-90s: {imp:>+5.1f}% | Switches: {n_switches}/{n_steps} ({switch_rate:.0f}%) {status}")
            
            if imp >= 10 and not self.reached:
                self.reached = True
                print(f"\n🏆 TARGET REACHED!")
        return True

FT90_REF = baseline_results['FT-90s']['mean_reward']
print(f"✅ Callback ready | Reference: FT-90s = {FT90_REF:.1f} | Target: {FT90_REF*1.1:.1f}")

✅ Callback ready | Reference: FT-90s = 799.6 | Target: 879.5


In [ ]:
# Cell 6: Initialize PPO Model - LONG RUN CONFIGURATION (1M Steps)
train_env = create_env()
eval_env = create_env()

# SCALING FOR 1 MILLION STEPS
TOTAL_STEPS = 1_000_000

# PPO Hyperparameters
# n_steps=2048 is standard for PPO (updates every n_steps * n_envs)
# ent_coef=0.01 encourages exploration (entropy regularization)
model = PPO("MlpPolicy", train_env, learning_rate=3e-4, 
            n_steps=2048, batch_size=64, n_epochs=10, 
            gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
            ent_coef=0.01, # Important for exploration in PPO
            verbose=0, device='cuda' if torch.cuda.is_available() else 'cpu')

# Eval freq: 1M steps / 200 points = 5000 steps
callback = ProgressCallback(eval_env, FT90_REF, target_pct=10.0, eval_freq=5000)
STATE = {'model': model, 'callback': callback, 'steps': 0, 'block': 0, 'done': False}

print(f"🚀 PPO ready on {model.device}")
print(f"   Target Steps: {TOTAL_STEPS} (1 Million)")
print(f"   Entropy Coef: 0.01 (Exploration)")
print(f"   Target: Beat FT-90s by +10%")


🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37
   🚦 Detected 8 signalized nodes from OSM data

   🔧 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))



🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
🏭 VICTORIA ISLAND CONFIG FACTORY - GLOBAL CONFIGURATION GENERATION
   📊 Loading topology from: /content/Code-traffic-flow/arz_model/data/fichier_de_travail_corridor_utf8.csv
   ✅ Loaded 70 edges from topology
   🔗 Building directed graph...
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37
   ✅ Graph built: 60 nodes, 70 edges
   🧠 Analyzing network structure (global reflection)...
   ✅ Network analysis complete:
      - Entry points: 4
      - Exit points: 4
      - Junctions: 15
      - Simple pass-through nodes: 37
   🚦 Detected 8 signalized nodes from OSM data
   🚦 D

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


🚀 DQN ready on cuda
   Exploration: 50% of training, final eps=0.1 (more exploration)
   Target: Beat FT-90s by +10%


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Block 1: Steps 0 → 100,000
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 1: 0 → 100k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 1, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_100k")
    print(f"✅ Block 1 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

📊 BLOCK 1: 0 → 10k


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size
70 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))

...

[  7000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[  8000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[  9000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[ 10000] R=  799.6 | vs FT-90s:  +0.0% | Switches: 0/30 (0%)

✅ Block 1 done | Steps: 10000 | Best: 799.6


In [ ]:
# Block 2: Steps 100k → 200k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 2: 100k → 200k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 2, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_200k")
    print(f"✅ Block 2 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

Output()

📊 BLOCK 2: 10k → 20k


[ 11000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[ 12000] R=  799.6 | vs FT-90s:  +0.0% | Switches: 1/30 (3%)

[ 13000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[ 14000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[ 15000] R=  758.7 | vs FT-90s:  -5.1% | Switches: 29/30 (97%)

[ 16000] R=  799.6 | vs FT-90s:  +0.0% | Switches: 1/30 (3%)

[ 17000] R=  799.6 | vs FT-90s:  +0.0% | Switches: 1/30 (3%)

[ 18000] R=  799.6 | vs FT-90s:  +0.0% | Switches: 1/30 (3%)

In [ ]:
# Block 3: Steps 200k → 300k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 3: 200k → 300k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 3, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_300k")
    print(f"✅ Block 3 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 4: Steps 300k → 400k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 4: 300k → 400k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 4, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_400k")
    print(f"✅ Block 4 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 5: Steps 400k → 500k (HALFWAY)
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 5: 400k → 500k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 5, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_500k")
    imp = ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100
    print(f"📈 HALFWAY: {STATE['steps']} steps | Best improvement: {imp:+.1f}%")

In [ ]:
# Block 6: Steps 500k → 600k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 6: 500k → 600k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 6, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_600k")
    print(f"✅ Block 6 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 7: Steps 600k → 700k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 7: 600k → 700k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 7, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_700k")
    print(f"✅ Block 7 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 8: Steps 700k → 800k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 8: 700k → 800k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 8, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_800k")
    print(f"✅ Block 8 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 9: Steps 800k → 900k
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 9: 800k → 900k")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 9, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_900k")
    print(f"✅ Block 9 done | Steps: {STATE['steps']} | Best: {STATE['callback'].best:.1f}")

In [ ]:
# Block 10 (FINAL): 900k → 1M
if STATE['done']: print("✅ Target reached, skipping")
else:
    print("📊 BLOCK 10 (FINAL): 900k → 1M")
    STATE['model'].learn(100000, callback=STATE['callback'], reset_num_timesteps=False, progress_bar=True)
    STATE['block'], STATE['steps'] = 10, STATE['callback'].num_timesteps
    STATE['done'] = STATE['callback'].reached
    STATE['model'].save(f"{SAVE_DIR}/model_FINAL")

imp = ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100
print(f"\n{'='*60}")
print(f"🏁 TRAINING COMPLETE")
print(f"   Total steps: {STATE['steps']}")
print(f"   Best reward: {STATE['callback'].best:.1f}")
print(f"   Improvement vs FT-90s: {imp:+.1f}%")
print(f"   Target (+10%): {'✅ ACHIEVED' if STATE['done'] else '❌ NOT ACHIEVED'}")
print(f"{'='*60}")

In [ ]:
# Final: Plot & Save Results
import matplotlib.pyplot as plt
import json

h = STATE['callback'].history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot([x['step'] for x in h], [x['reward'] for x in h], 'b-o', markersize=3)
ax1.axhline(y=FT90_REF, color='r', linestyle='--', label='FT-90s')
ax1.axhline(y=baseline_results['FT-30s']['mean_reward'], color='g', linestyle=':', label='FT-30s')
ax1.set_xlabel('Steps'); ax1.set_ylabel('Reward'); ax1.legend(); ax1.set_title('Learning Curve')

ax2.plot([x['step'] for x in h], [x['improvement'] for x in h], 'b-o', markersize=3)
ax2.axhline(y=10, color='g', linestyle='--', label='Target +10%')
ax2.axhline(y=0, color='r', linestyle='-', alpha=0.5)
ax2.set_xlabel('Steps'); ax2.set_ylabel('Improvement (%)'); ax2.legend(); ax2.set_title('Improvement vs FT-90s')

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/stage3_results.png', dpi=150)
plt.show()

# Save JSON
results = {'baselines': baseline_results, 'history': h, 'best': STATE['callback'].best, 
           'improvement': ((STATE['callback'].best - FT90_REF) / abs(FT90_REF)) * 100}
with open(f'{SAVE_DIR}/stage3_results.json', 'w') as f: json.dump(results, f, indent=2)
print(f"📁 Saved: {SAVE_DIR}/stage3_results.png, {SAVE_DIR}/stage3_results.json")